In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [76]:
import numpy as np
from sympy import simplify
from matplotlib import pyplot as plt
import scipy.io
from math import sqrt

np.random.seed(10)

In [77]:
def get_homogenous(R, P):
    p = P.copy().reshape(3)
    r = R.copy().reshape(3,3)
    return np.array([[r[0,0], r[0,1], r[0,2], p[0]],
                     [r[1,0], r[1,1], r[1,2], p[1]],
                     [r[2,0], r[2,1], r[2,2], p[2]],
                     [0     , 0     , 0     , 1]], dtype='float').reshape(4,4)


def translation_x(l):
    return np.array([[1,0,0, l],
                     [0,1,0, 0],
                     [0,0,1, 0],
                     [0,0,0, 1]], dtype='float')

def translation_y(l):
    return np.array([[1,0,0, 0],
                     [0,1,0, l],
                     [0,0,1, 0],
                     [0,0,0, 1]], dtype='float')

def translation_z(l):
    return np.array([[1,0,0, 0],
                     [0,1,0, 0],
                     [0,0,1, l],
                     [0,0,0, 1]], dtype='float')

def rotation_x(theta):
    return np.array([[1,         0,          0, 0],
                     [0,np.cos(theta),-np.sin(theta), 0],
                     [0,np.sin(theta), np.cos(theta), 0],
                     [0,         0,          0, 1]], dtype='float')


def rotation_y(theta):
    return np.array([[np.cos(theta) ,0,np.sin(theta), 0],
                     [0          ,1,         0, 0],
                     [-np.sin(theta),0,np.cos(theta), 0],
                     [0          ,0,         0, 1]], dtype='float')

def rotation_z(theta):
    return np.array([[np.cos(theta),-np.sin(theta),0, 0],
                     [np.sin(theta), np.cos(theta),0, 0],
                     [0         ,0          ,1, 0],
                     [0         ,0          ,0, 1]], dtype='float')


def rotation_x3(theta):
    return np.array([[1,         0,          0],
                     [0,np.cos(theta),-np.sin(theta)],
                     [0,np.sin(theta), np.cos(theta)]], dtype='float')


def rotation_y3(theta):
    return np.array([[np.cos(theta) ,0,np.sin(theta)],
                     [0          ,1,         0],
                     [-np.sin(theta),0,np.cos(theta)]], dtype='float')

def rotation_z3(theta):
    return np.array([[np.cos(theta),-np.sin(theta),0],
                     [np.sin(theta), np.cos(theta),0],
                     [0         ,0          ,1]], dtype='float')



def dtranslation_x(l):
    return np.array([[0,0,0, 1],
                     [0,0,0, 0],
                     [0,0,0, 0],
                     [0,0,0, 0]], dtype='float')

def dtranslation_y(l):
    return np.array([[0,0,0, 0],
                     [0,0,0, 1],
                     [0,0,0, 0],
                     [0,0,0, 0]], dtype='float')

def dtranslation_z(l):
    return np.array([[0,0,0, 0],
                     [0,0,0, 0],
                     [0,0,0, 1],
                     [0,0,0, 0]], dtype='float')

def drotation_x(theta):
    return np.array([[0,         0,          0, 0],
                     [0,-np.sin(theta), -np.cos(theta), 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0,         0,          0, 0]], dtype='float')


def drotation_y(theta):
    return np.array([[-np.sin(theta), 0,  np.cos(theta), 0],
                     [0          ,0,         0, 0],
                     [-np.cos(theta), 0, -np.sin(theta), 0],
                     [0          ,0,         0, 0]], dtype='float')

def drotation_z(theta):
    return np.array([[-np.sin(theta),-np.cos(theta),0, 0],
                     [ np.cos(theta), -np.sin(theta),0, 0],
                     [0         ,0          ,0, 0],
                     [0         ,0          ,0, 0]], dtype='float')


def get_rotation(H):
    return H[:3,:3]

def get_position(H):
    return H[:3,3]

def calc_error(H1, H2):
    shape = np.array(H1).shape
    error = 0
    for i in range(shape[0]):
        for j in range(shape[1]):
            error += abs(H1[i,j] - H2[i,j])
    return error

def check_diff(eq1, eq2):
    return simplify(eq1 - eq2) == 0

def print_matrix(f):
        print(f"Homogeneous Matrix:\n{f}")
        print("Rotation:\n", get_rotation(f))
        print("Position:\n", get_position(f)) 

def pos2hom(pos):
    hom = np.zeros((4,4))
    hom[:3,3] = pos.T
    hom[3,3] = 1
    return hom

def plot_u(u, dt=1/100, title="Control Input", time=None):
    
    u = np.array(u).reshape(len(u), 2)
    time = np.linspace(0, dt*len(u), len(u)) if time is None else time

    fig, ax = plt.subplots(1)
    ax.plot(time, u)
    ax.set_xlabel("Time - seconds")
    ax.set_ylabel("u - (torque -- N.m)")
    ax.legend(["Joint1", "Joint2"], loc="upper left", bbox_to_anchor=(1, 1))
    ax.set_title("Control - Torques on Joints")
    
    fig.suptitle(title, fontsize=12)
    plt.tight_layout()
    plt.show()

def plot_trajectory(traj, dt=1/100, title="Trajectory", time=None):
    q, dq, ddq = traj[:]
    
    q = np.array(q).squeeze()
    dq = np.array(dq).squeeze()
    ddq = np.array(ddq).squeeze()
    
    time = np.linspace(0, dt*len(q), len(q)) if time is None else time

    fig, axs = plt.subplots(3,1)
    axs[0].plot(time, q)
    axs[0].set_xlabel("Time - seconds")
    axs[0].set_ylabel("q - rad")
    axs[0].legend(["Joint1", "Joint2"], loc="upper left", bbox_to_anchor=(1, 1))
    axs[0].set_title("Position")

    axs[1].plot(time, dq)
    axs[1].set_xlabel("Time - seconds")
    axs[1].set_ylabel("dq - rad/sec")
    axs[1].legend(["Joint1", "Joint2"], loc="upper left", bbox_to_anchor=(1, 1))
    axs[1].set_title("Velocity")
    
    axs[2].plot(time, ddq)
    axs[2].set_xlabel("Time - seconds")
    axs[2].set_ylabel("dq - rad/sec^2")
    axs[2].legend(["Joint1", "Joint2"], loc="upper left", bbox_to_anchor=(1, 1))
    axs[2].set_title("Acceleration")

    
    fig.suptitle(title, fontsize=12)
    plt.tight_layout()
    plt.show()
    
# https://stackoverflow.com/questions/36915774/form-numpy-array-from-possible-numpy-array
def skew(x):
    if (isinstance(x,np.ndarray) and len(x.shape)>=2):
        return np.array([[0, -x[2][0], x[1][0]],
                        [x[2][0], 0, -x[0][0]],
                        [-x[1][0], x[0][0], 0]]).reshape((3,3))
    else:
        return np.array([[0, -x[2], x[1]],
                        [x[2], 0, -x[0]],
                        [-x[1], x[0], 0]]).reshape((3,3))


tx = translation_x
ty = translation_y
tz = translation_z

rx = rotation_x
ry = rotation_y
rz = rotation_z

drx = drotation_x
dry = drotation_y
drz = drotation_z

dtx = dtranslation_x
dty = dtranslation_y
dtz = dtranslation_z

In [78]:
class FANUC_R_2000i_configs:
    # TODO: change static methods to class variables
    @staticmethod
    def get_links_dimensions():
        return [400,25,560,25,515,90]

    @staticmethod
    def get_joints_limits():
        deg = [(-170, 170), (-190, 45), (-120, 156), (-185, 185), (-120, 120), (-350, 350)]
        rad = []
        for (i,j) in deg:
            rad.append(((i*np.pi/180), (j*np.pi/180)))
        return rad

class FANUC_R_2000i:
    def __init__(self, T_base=None, T_tool=None):
        self.num_joints = 6
        self.robot_configs = FANUC_R_2000i_configs
        self.d = self.robot_configs.get_links_dimensions()
        self.joint_limits = self.robot_configs.get_joints_limits()
        self.T_base = translation_x(0) if T_base is None else T_base
        self.T_tool = translation_x(0) if T_tool is None else T_tool

    def get_T_robot_reducible(self, q, pi):
        T_robot = rz(q[0]) @ tx(self.d[1]+pi[0]) @ ty(pi[1]) @ rx(pi[2]) @ ry(q[1]+pi[3]) @ tx(pi[4]) @ rx(pi[5]) @ rz(pi[6]) @ ry(q[2]+pi[7]) @ tx(self.d[5]+pi[8]) @ tz(self.d[4]+pi[9]) @ rz(pi[10]) @ rx(q[3]+pi[11]) @ ty(pi[12]) @ tz(pi[13]) @ rz(pi[14]) @ ry(q[4] + pi[15]) @ tz(pi[16]) @ rz(pi[17]) @ rx(q[5])
        return T_robot


In [79]:
class Jacobian:
    def __init__(self, robot, T_base=None, T_tool=None):
        self.T_base = tx(0) if T_base is None else T_base
        self.T_tool = tx(0) if T_tool is None else T_tool
        self.robot = robot
        self.d = self.robot.robot_configs.get_links_dimensions()

    def _get_jacobian_column(self, dT):
        J = np.zeros((6,1))
        
        J[:3, 0] = dT[:3,3]
        J[3,0] = dT[2,1]
        J[4,0] = dT[0,2]
        J[5,0] = dT[1,0]
        return J.squeeze()

    def calc_identification_jacobian(self, T_base, T_tool, q, pi, pi_0, num_unknown_parameters=18):
        J = np.zeros((6,num_unknown_parameters))
        # Reducible Kinmatic Model: pi_0 is the nomial pi for the unknown parameters
        T = T_base @ self.robot.get_T_robot_reducible(q, pi) @ T_tool
        
        To_inv = np.eye(4)
        To_inv[:3,:3] = np.linalg.inv(T[:3,:3])
        
        dT = T_base @ rz(q[0]) @ dtx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,0] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ dty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,1] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ drx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,2] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ dry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,3] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ dtx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,4] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ drx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,5] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ drz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,6] = self._get_jacobian_column(dT)
        
        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ dry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,7] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ dtx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,8] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ dtz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,9] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ drz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,10] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]) @ drx(pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,11] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ dty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,12] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ dtz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,13] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ drz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,14] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ dry(pi_0[15]) @ tz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,15] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ dtz(pi_0[16]) @ rz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,16] = self._get_jacobian_column(dT)

        dT = T_base @ rz(q[0]) @ tx(pi_0[0]) @ ty(pi_0[1]) @ rx(pi_0[2]) @ ry(q[1]) @ ry(pi_0[3]) @ tx(pi_0[4]) @ rx(pi_0[5]) @ rz(pi_0[6]) @ ry(q[2]) @ ry(pi_0[7]) @ tx(pi_0[8]) @ tz(pi_0[9]) @ rz(pi_0[10]) @ rx(q[3]+pi_0[11]) @ ty(pi_0[12]) @ tz(pi_0[13]) @ rz(pi_0[14]) @ ry(q[4]) @ ry(pi_0[15]) @ tz(pi_0[16]) @ drz(pi_0[17]) @ rx(q[5]) @ T_tool @ To_inv
        J[:,17] = self._get_jacobian_column(dT)

        return J

In [80]:
class Calibration:
    def __init__(self, robot,
                       dataset_file="calibration_dataset.mat",
                       num_configs=24, num_samples=10, num_reference_points=3,
                       reference_points_tags=["mA", "mB", "mC"],
                       save_postfix=""):
        self.dataset_file = dataset_file
        self.num_configs = num_configs
        self.num_samples = num_samples
        self.num_reference_points = num_reference_points
        self.reference_points_tags = reference_points_tags
        self.num_joints = robot.num_joints
        self.dimension = 3
        self.num_unknown_parameters = 4+8+6 # Only the robot # 27: including T_base and T_tool
        self.robot = robot
        self.save_postfix=save_postfix
        
        self.jacobian = Jacobian(self.robot)
                
        self.d = robot.robot_configs.get_links_dimensions()

        # self.pi_0 = np.zeros((self.num_unknown_parameters, 1))
        self.pi_0 = np.array([self.d[1], 0,0,0,0,0,0,0, self.d[5], self.d[4], 0, 0,0,0,0,0,0,0], dtype=np.float).reshape((self.num_unknown_parameters, 1)) 
        
        self.visualization_radius = {"node":0.003}
        self.visualization_scale = 0.1
        self.visualization_rate = 5

        
        self.read_mat_file()
        self.splitter()
        
    def get_dataset_raw(self):
        return self.dataset_raw
    
    def read_mat_file(self):
        self.dataset_raw = scipy.io.loadmat(self.dataset_file)
        
    
    def _rescale(self, vec):
        return [vec[0]*1000, vec[1]*1000, vec[2]*1000]
    def splitter(self):
        self.configruations = np.empty((self.num_configs, self.num_samples, self.num_joints, 1))
        self.dataset = np.empty((self.num_configs, self.num_samples, self.num_reference_points, self.dimension))
        for i in range(self.num_configs):
            for j in range(self.num_samples):
                self.configruations[i, j] = np.array(self.dataset_raw["q"][j+self.num_samples*i]).reshape((self.num_joints, 1))
                for k in range(self.num_reference_points):
                    original_scale = self.dataset_raw[self.reference_points_tags[k]][j+self.num_samples*i]
                    self.dataset[i, j, k] = self._rescale(original_scale)
                    # print(f"{i}, {j}, {k} <- {self.reference_points_tags[k]}{j+self.num_samples*i}")
        # return(self.dataset[0,0])

    def get_std_config(self, config=None, config_idx=None):
        if(config_idx is not None):
            config = self.dataset[config_idx]
        if(config is None):
            raise AttributeError('Please provide parameter config or config_idx')
        std = []
        # print(config[:,0]) # 10 repetitions for mA in the configuration
        for i in range(self.num_reference_points):
            std.append(np.std(config[:,i], axis=0))
        return std
        
    def get_mean_config(self, config=None, config_idx=None):
        if(config_idx is not None):
            config = self.dataset[config_idx]
        if(config is None):
            raise AttributeError('Please provide parameter config or config_idx')
        mean = []
        for i in range(self.num_reference_points):
            mean.append(np.mean(config[:,i], axis=0))
        return mean      

    def visualize(self):
        vis = visual.RobotVisualization_vpython(rate=self.visualization_rate,
                                                scale=self.visualization_scale,
                                                radius=self.visualization_radius)
        frame = []
        for i, config in enumerate(self.dataset):
            # print(f"{i+1}th Configuration")
        # for i in range(len(self.dataset)):
        # config = self.dataset[0]
            for sample in config:
                for pos in sample:       
                    frame.append(["node", pos])
                # print(f"Std for mA, mB, mC: {self.get_std_config(config_idx=0)}")
                # print(f"Mean: {self.get_mean_config(config_idx=0)}")
            # print(f"Std for mA, mB, mC:\n{self.get_std_config(config=config)}")
            # print(f"Mean:\n{self.get_mean_config(config=config)}")
            # print("----------------------------------------------------------------")

        while True:
            vis.render_frame(frame, axis=True)
            # time.sleep(5)
            
            
    # Use q, pi (error) in order to base and tool tranformations             
    def _step1(self):                    
        sum1 = np.zeros((6+3*self.num_reference_points,6+3*self.num_reference_points))
        sum2 = np.zeros((6+3*self.num_reference_points,1))
        # sum
        # print(sum2)
        for i in range(self.num_configs):
            for j in range(self.num_samples):
                delta_p_i = np.empty((3*self.num_reference_points,1))
                q = self.configruations[i, j].copy()
                self.T_robot = self.robot.get_T_robot_reducible(q, self.pi)
                p_robot_i = get_position(self.T_robot).copy().reshape((3,1))
                R_robot_i = get_rotation(self.T_robot).copy().reshape((3,3))
                # Calculate A matrix
                A = np.empty((3*self.num_reference_points, 6+3*self.num_reference_points))
                for k in range(self.num_reference_points):
                    A[k*3:k*3+3, :3] = np.eye(3)
                    A[k*3:k*3+3, 3:6] = skew(p_robot_i).T
                    for kk in range(self.num_reference_points):
                        if(kk == k):
                            A[k*3:k*3+3, 6+kk*3:6+kk*3+3] = R_robot_i

                        else:
                            A[k*3:k*3+3, 6+kk*3:6+kk*3+3] = np.zeros((3,3))
                    # print(self.dataset[i,j,k])
                    # print(get_position(self.T_base@self.T_robot@self.T_tool[k]))
                    delta_p_i_j = self.dataset[i,j,k].copy() - get_position(self.T_robot).copy()
                    delta_p_i[k*3:k*3+3] = delta_p_i_j.reshape(3,1)
                # Calculate the 1st summation
                sum1 += A.T @ A
                # Calulate the 2nd summation
                sum2 += A.T @ delta_p_i
                
        # Apply the formula and get p_base, phi_base, u_tool^1, ..., u_tool^num_reference_points
        tmp = np.linalg.inv(sum1) @ sum2  # shape: ((6+3n)x1)
        p_base = np.array(tmp[:3, 0]).reshape((3,1))
        phi_base = np.array(tmp[3:6, 0]).reshape((3,1))
        # From p_base and phi_base get T_base
        R_base = skew(phi_base) + np.eye(3)
        T_base = get_homogenous(R_base, p_base)
        T_tool = np.empty((self.num_reference_points, 4, 4))
        # From u_tool^j get p_tool^j st. j=num_reference_points
        # From p_tool^j get T_tool^j -> get T_tool
        for i in range(self.num_reference_points):
            u_tool_j = np.array(tmp[6+i*3:6+(i+1)*3, 0]).reshape((3,1))
            p_tool_j = R_base.T @ u_tool_j
            T_tool_j = get_homogenous(np.eye(3), p_tool_j)
            T_tool[i] = T_tool_j
        return T_base, T_tool
    
    # Use T_base, T_tool, q, pi
    def _step2(self):
        sum1 = np.zeros((self.num_unknown_parameters,self.num_unknown_parameters))
        sum2 = np.zeros((self.num_unknown_parameters,1))
        for i in range(self.num_configs):
            for j in range(self.num_samples):
                q = self.configruations[i, j].copy()
                self.T_robot = self.robot.get_T_robot_reducible(q, self.pi)
                # Calculate identification Jacobian J_pi
                for k in range(self.num_reference_points):
                    jacobian_pi = self.jacobian.calc_identification_jacobian(T_base=self.T_base, T_tool=self.T_tool[k], q=q, pi=self.pi, pi_0=self.pi_0)
                    jacobian_pi_jp = jacobian_pi[:3].reshape((3, self.num_unknown_parameters))
                    delta_p_i_j = (self.dataset[i,j,k].copy() - get_position(self.T_robot).copy()).reshape((3,1))
                    
                    # Calculate the 1st summation
                    sum1 += jacobian_pi_jp.T @ jacobian_pi_jp
                    # Calculate the 2nd summation
                    sum2 += jacobian_pi_jp.T @ delta_p_i_j
                    
        # Apply the formula and get delta_pi
        delta_pi = np.linalg.inv(sum1) @ sum2
        
        return delta_pi
    
    def _terminamtion_criteria(self, delta_pi, epsilon):
        stopping_sum = 0
        for i in range(self.num_configs):
            for j in range(self.num_samples):
                q = self.configruations[i, j].copy()
                self.T_robot = self.robot.get_T_robot_reducible(q, self.pi)
                # Calculate identification Jacobian J_pi
                for k in range(self.num_reference_points):
                    jacobian_pi = self.jacobian.calc_identification_jacobian(T_base=self.T_base, T_tool=self.T_tool[k], q=q, pi=self.pi, pi_0=self.pi_0)
                    jacobian_pi_jp = jacobian_pi[:3].reshape((3, self.num_unknown_parameters))
                    delta_p_i_j = (self.dataset[i,j,k].copy() - get_position(self.T_robot).copy()).reshape((3,1))
                    term = jacobian_pi_jp.dot(delta_pi) - delta_p_i_j
                    stopping_sum += term.T @ term
        print(np.linalg.norm(stopping_sum))
        if(np.linalg.norm(stopping_sum) < epsilon):
            return True
        return False
    
    def calibrate(self, max_num_steps=101, alpha=0.001, epsilon=1e-8):
        steps = 0
        # define pi
        self.pi = self.pi_0
        self.T_base = np.eye(4)
        self.T_tool = np.zeros((self.num_reference_points, 4, 4))
        for i in range(self.num_reference_points):
            self.T_tool[i,:,:] = np.eye(4)
        while True:
            print(f"{steps+1}th iteration:")
            # return
            self.T_base, self.T_tool = self._step1()
            # print("T_base")
            # print(self.T_base)
            # print("T_tool")
            # print(self.T_tool)
            delta_pi = self._step2()
            self.pi += alpha*delta_pi

            if(steps % 10 == 0):
                print(f"Pi:\n{self.pi}")
                np.save(f"pi{self.save_postfix}.npy", self.pi)
                print(f"T_base:\n{self.T_base}")
                np.save(f"T_base{self.save_postfix}.npy", self.T_base)
                print(f"T_tool:\n{self.T_tool}")
                np.save(f"T_tool{self.save_postfix}.npy", self.T_tool)
            
            steps += 1
            if(steps >= max_num_steps or self._terminamtion_criteria(delta_pi, epsilon)):
                break
            
        print("-------------------------")
        print(self.pi)
        
    def RMS_report(self, pi=None, T_base=None, T_tool=None):
        pi = self.pi if pi is None else pi
        T_base = self.T_base if T_base is None else T_base
        T_tool = self.T_tool if T_tool is None else T_tool
        max_dist = 0
        error = 0 #sqrt(1/n sum(^2))
        error_coordinates = np.zeros(self.dimension)
        max_err_coordinates = np.zeros(self.dimension)
        for i in range(self.num_configs):
            for j in range(self.num_samples):
                q = self.configruations[i, j].copy()
                for k in range(self.num_reference_points):
                    T_robot = self.robot.get_T_robot_reducible(q, pi)
                    T = T_base @ T_robot @ T_tool[k]
                    new_pos = get_position(T).copy()
                    pos = self.dataset[i, j, k].copy()
                    # print(new_pos - pos)

                    dist = np.linalg.norm(new_pos - pos)
                    max_dist = max(max_dist, dist)
                    error += dist**2
                    for e in range(self.dimension):
                        err = abs(new_pos[e] - pos[e])
                        error_coordinates[e] += err
                        max_err_coordinates[e] = max(max_err_coordinates[e], err)
        N = (self.num_configs*self.num_samples*self.num_reference_points)
        error = sqrt(error/N)
        print(f"RMS Error: {error}")
        print(f"Max Distance error (mm): {max_dist}")
        print("-------------------------")
        
        tags = ['x', 'y', 'z']
        for e in range(self.dimension):
            error_coordinates[e] = sqrt(error_coordinates[e]/N)
            print(f"RMS Error for {tags[e]}-coordinate: {error_coordinates[e]}")
            print(f"Max error for {tags[e]}-coordinate (mm): {max_err_coordinates[e]}")
            print("-------------------------")
        return error

# robot = FANUC_R_2000i()
# calib = Calibration(robot=robot, dataset_file="/content/gdrive//MyDrive/calibration_dataset.mat")
# calib.calibrate(alpha=0.05)

In [ ]:
robot = FANUC_R_2000i()
calib = Calibration(robot=robot, dataset_file="/content/gdrive//MyDrive/calibration_dataset.mat")
calib.calibrate(alpha=1)
postfix = ""#" (0)"
    
T_base = np.load(f"T_base{postfix}.npy")
# T_base = translation_x(0)

T_tool = np.load(f"T_tool{postfix}.npy")
# T_tool[0,:3,3] = np.zeros((3,))
# T_tool[1,:3,3] = np.zeros((3,))
# T_tool[2,:3,3] = np.zeros((3,))

pi = np.load(f"pi{postfix}.npy")
# pi = np.zeros((18, 1))
# pi = np.array([d[1], 0,0,0,0,0,0,0, d[5], d[4], 0, 0,0,0,0,0,0,0], dtype=np.float).reshape((18, 1)) 

np.set_printoptions(precision=3, suppress=True,)
print(f"Pi:\n{np.array2string(pi, separator=', ')}")
print(f"T_base:\n{np.array2string(T_base, separator=', ')}")
print(f"T_tool:\n{np.array2string(T_tool, separator=', ')}")
calib.RMS_report(pi=pi, T_base=T_base, T_tool=T_tool)

1th iteration:
